<a href="https://colab.research.google.com/github/ngockhanh5110/nlp-vietnamese-text-summarization/blob/main/notebooks/training_vietnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
OUTPUT_DIR = '/kaggle/working/training/'

In [2]:
%%capture
!pip install transformers
!pip install datasets
!pip install gdown
!pip install rouge_score
!pip install evaluate

In [3]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *
import gdown
import datasets
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,EncoderDecoderModel
import transformers
import os
os.environ['WANDB_DISABLED'] = 'true'

## Processing data

In [4]:
gdown.download('https://drive.google.com/file/d/13FF_pNJth3GeKHQEzmD-voywcTmjAFIb/view?usp=sharing','/kaggle/working/train_data.csv',fuzzy=True)
gdown.download('https://drive.google.com/file/d/1a3xuGQA1Ztpti8kzfBtO4e0zSFbUFECq/view?usp=sharing','/kaggle/working/val_data.csv',fuzzy=True)
gdown.download('https://drive.google.com/file/d/1S-POATiC3cbMEUuEmkYOf_NE55e2MErU/view?usp=sharing','/kaggle/working/test_data.csv',fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=13FF_pNJth3GeKHQEzmD-voywcTmjAFIb
From (redirected): https://drive.google.com/uc?id=13FF_pNJth3GeKHQEzmD-voywcTmjAFIb&confirm=t&uuid=bccba3f6-5e6a-49fc-8121-756cebbb7679
To: /kaggle/working/train_data.csv
100%|██████████| 218M/218M [00:01<00:00, 207MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1a3xuGQA1Ztpti8kzfBtO4e0zSFbUFECq
To: /kaggle/working/val_data.csv
100%|██████████| 46.7M/46.7M [00:00<00:00, 229MB/s]
Downloading...
From: https://drive.google.com/uc?id=1S-POATiC3cbMEUuEmkYOf_NE55e2MErU
To: /kaggle/working/test_data.csv
100%|██████████| 46.9M/46.9M [00:00<00:00, 222MB/s]


'/kaggle/working/test_data.csv'

In [5]:
train_df = pd.read_csv('/kaggle/working/train_data.csv')
val_df = pd.read_csv('/kaggle/working/val_data.csv')
test_df = pd.read_csv('/kaggle/working/test_data.csv')

### **Data Preprocessing**

In [6]:
tokenizer =  AutoTokenizer.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("vinai/phobert-base", "vinai/phobert-base", tie_encoder_decoder=False)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['roberta.encoder.layer.0.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.0.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.query.weight', 'roberta.encoder.layer.0.crossattention.self.value.bias', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.1.crossattention.output.dense.bias', 'roberta.encoder.layer.1.crossattention.output.dense.weight', 'roberta.encod

In [8]:
train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [9]:
train_set = train_data.map(lambda  x:
                           {'input_ids': tokenizer(x['original'], padding="max_length", truncation=True, max_length=256).input_ids,
                               'labels': tokenizer(x['summary'], padding="max_length", truncation=True, max_length=32).input_ids,
                           })
val_set = val_data.map(lambda  x:
                            {'input_ids': tokenizer(x['original'], padding="max_length", truncation=True, max_length=256).input_ids,
                               'labels': tokenizer(x['summary'], padding="max_length", truncation=True, max_length=32).input_ids,
                           })
test_set = test_data.map(lambda  x:
                            {'input_ids': tokenizer(x['original'], padding="max_length", truncation=True, max_length=256).input_ids,
                               'labels': tokenizer(x['summary'], padding="max_length", truncation=True, max_length=32).input_ids,
                           })

Map:   0%|          | 0/105418 [00:00<?, ? examples/s]

Map:   0%|          | 0/22642 [00:00<?, ? examples/s]

Map:   0%|          | 0/22644 [00:00<?, ? examples/s]

In [10]:
roberta_shared.config

EncoderDecoderConfig {
  "decoder": {
    "_name_or_path": "vinai/phobert-base",
    "add_cross_attention": true,
    "architectures": [
      "RobertaForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 2,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,
 

### **Warm-starting the Encoder-Decoder Model**

In [11]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id
roberta_shared.config.eos_token_id = tokenizer.eos_token_id
roberta_shared.config.pad_token_id = tokenizer.pad_token_id
# sensible parameters for beam search
# set decoding params
roberta_shared.config.max_length = 32
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

In [13]:
# load rouge for validation
rouge = evaluate.load('rouge')

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    
    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
batch_size = 32
training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=200,  # set to 2000 for full training
    save_steps=10000,  # set to 500 for full training
    eval_steps=10000,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=50,
    fp16=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer = tokenizer
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/opt/conda/lib/python3.10/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.

Step,Training Loss
